In [1]:
import requests
import base64
import json

In [2]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

# OAuth Authorization

In [3]:
import flask
import urllib.parse as urlparse

code = None
app = flask.Flask(__name__)

scopes = " ".join([
    "user-read-playback-state"
    ,"user-read-currently-playing"
    ,"playlist-read-private"
    ,"playlist-read-collaborative"
    ,"user-read-playback-position"
    ,"user-top-read"
    ,"user-read-recently-played"
    ,"user-library-read"
    ,"user-read-email"
    ,"user-read-private"])

@app.route("/")
def index():
    
    user_auth_url = 'https://accounts.spotify.com/authorize?' + urlparse.urlencode({
        'client_id': client_id,
        'response_type': 'code',
        'scope': scopes,
        'redirect_uri': redirect_url,
    })  
    return flask.redirect(user_auth_url)

@app.route("/callback")
def callback():
    code = flask.request.args.get('code')
    if code is None:
        err = flask.request.args.get('error')
        shutdown_server()
        return f"Authorization failed with the following error: {err}"
    else:
        with open('auth_code.txt','w+') as file:
            file.write(code)

        shutdown_server()
        return "<h1> Authorization successful. Please close this window. </h1>"

def shutdown_server():
    func = flask.request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

if __name__ == "__main__":
    app.run(host="localhost", port=5050)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Feb/2024 00:18:49] "GET / HTTP/1.1" 302 -
/var/folders/_m/s8jbyc4n0yjb3n2vnvp2pc4h0000gn/T/ipykernel_13314/3966248663.py:48: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


# Get Token

In [4]:
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    auth_api_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(auth_api_url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    return(token)

In [18]:
def get_auth_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_encoded = str(base64.b64encode(auth_bytes), "utf-8")
    
    with open('auth_code.txt','r+') as file:
        auth_code = file.read()
    
    token_endpoint = "https://accounts.spotify.com/api/token"
    data = {
        "grant_type":"authorization_code",
        "code":auth_code,
        "redirect_uri":redirect_url,
    }
    headers = {
        "Authorization": "Basic " + auth_encoded,
        "Content-type": "application/x-www-form-urlencoded"
    }
    
    data = {"grant_type": "client_credentials"}
    result = requests.post(token_endpoint, headers=headers, data=data)
    json_results = json.loads(result.content)
    return json_result